In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from yahoo_historical import Fetcher
from statsmodels.tsa.api import ExponentialSmoothing

In [2]:
import pandas as pd
aapl = pd.read_csv('news_aapl.csv')
aapl = aapl.append(pd.read_csv('news_aapl2.csv'))
aapl = aapl.reset_index(drop=True)

In [3]:
aapl.head()

,text,Date,name
0,China's @Huawei celebrates @Nikkei iPhone Ban ...,2019-01-02,AppleInsider
1,Is $AAPL in any real trouble? (yes.) from me a...,2019-01-02,Yahoo Finance
2,"Editorial: Based on headlines, you'd think tha...",2019-01-03,AppleInsider
3,LIVE: $AAPL shares are down over 8% ahead of t...,2019-01-03,Yahoo Finance
4,"Highlight: ""We can’t emphasize enough how impo...",2019-01-03,Yahoo Finance


In [4]:
aapl.shape

(278, 3)

In [5]:
aapl2 = aapl.drop_duplicates()
aapl2.shape

(272, 3)

In [6]:
aapl[aapl.duplicated()==True]

,text,Date,name
35,Highlight: $AAPL CEO Tim Cook takes home a $12...,2019-01-09,Yahoo Finance
70,Highlight: @danielhowley on $AAPL simplifying ...,2019-02-20,Yahoo Finance
82,Highlight: “We expect $AAPL to continue doing ...,2019-03-12,Yahoo Finance
92,"Highlight: ""It's really all about the video-st...",2019-03-22,Yahoo Finance
135,Highlight: @AlbionFinGrp CIO Jason Ware on his...,2019-04-29,Yahoo Finance
234,Highlight: Bob Iger split from $AAPL ahead of ...,2019-09-16,Yahoo Finance


In [7]:
aapl.iloc[34,::]

text    Highlight: $AAPL CEO Tim Cook takes home a $12...
Date                                           2019-01-09
name                                        Yahoo Finance
Name: 34, dtype: object

In [8]:
aapl=aapl2

In [9]:
aapl.Date = pd.to_datetime(aapl.Date)
print(aapl.Date.min())
print(aapl.Date.max())

2019-01-02 00:00:00
2019-12-27 00:00:00


In [10]:
ticker = 'AAPL'
start = [2019,1,1]

df = Fetcher(ticker, start).getHistorical()	
#df = df[['Date','Close']]
df.Date = pd.to_datetime(df.Date)

In [11]:
df['close_tmr'] = df.Close.shift(-1)
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,close_tmr
0,2019-01-02,154.889999,158.850006,154.229996,157.919998,155.214005,37039700,142.190002
1,2019-01-03,143.979996,145.720001,142.000000,142.190002,139.753540,91312200,148.259995
2,2019-01-04,144.529999,148.550003,143.800003,148.259995,145.719513,58607100,147.929993
3,2019-01-07,148.699997,148.830002,145.899994,147.929993,145.395203,54777800,150.750000
4,2019-01-08,149.559998,151.820007,148.520004,150.750000,148.166870,41025300,153.309998


In [12]:
df['chg'] = df.close_tmr>df.Close
df['chg'] = df.chg.astype(int)
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,close_tmr,chg
0,2019-01-02,154.889999,158.850006,154.229996,157.919998,155.214005,37039700,142.190002,0
1,2019-01-03,143.979996,145.720001,142.000000,142.190002,139.753540,91312200,148.259995,1
2,2019-01-04,144.529999,148.550003,143.800003,148.259995,145.719513,58607100,147.929993,0
3,2019-01-07,148.699997,148.830002,145.899994,147.929993,145.395203,54777800,150.750000,1
4,2019-01-08,149.559998,151.820007,148.520004,150.750000,148.166870,41025300,153.309998,1


In [13]:
df.shape

(317, 9)

In [14]:
aapl.shape

(272, 3)

In [15]:
df.Date = pd.to_datetime(df.Date)

In [16]:
aapl = aapl.merge(df[['Date','chg']],on='Date')
aapl.shape

(226, 4)

In [17]:
aapl.head()

,text,Date,name,chg
0,China's @Huawei celebrates @Nikkei iPhone Ban ...,2019-01-02,AppleInsider,0
1,Is $AAPL in any real trouble? (yes.) from me a...,2019-01-02,Yahoo Finance,0
2,"Editorial: Based on headlines, you'd think tha...",2019-01-03,AppleInsider,1
3,LIVE: $AAPL shares are down over 8% ahead of t...,2019-01-03,Yahoo Finance,1
4,"Highlight: ""We can’t emphasize enough how impo...",2019-01-03,Yahoo Finance,1


In [18]:
from sklearn.model_selection import train_test_split
tr,te = train_test_split(aapl, test_size=0.2, random_state=0)
print(tr.shape)
print(te.shape)

(180, 4)
(46, 4)


In [19]:
text = [a for a in tr.text]
len(text)

180

In [20]:
text[0]

'Apple’s COO gives blistering testimony on Qualcomm’s "onerous demands" over cellular standard-essential #patents\n$AAPL $QCOM @danieleran https://t.co/VMoYzzALR6 https://t.co/AVJxbyvsZP'

In [21]:
a=text[0]

In [22]:
' '.join(filter(lambda x:x[0:4]!='http', a.split()))

'Apple’s COO gives blistering testimony on Qualcomm’s "onerous demands" over cellular standard-essential #patents $AAPL $QCOM @danieleran'

In [38]:
text = [' '.join(filter(lambda x:x[0:4]!='http', a.split())) for a in text]
text[0]

'Apple’s COO gives blistering testimony on Qualcomm’s "onerous demands" over cellular standard-essential #patents $AAPL $QCOM @danieleran'

In [39]:
import nltk
#nltk.download()
#nltk.download('stopwords')
#nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
import string

stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\hadit\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [40]:
text_tokens = [nltk.word_tokenize(text[i]) for i in range(len(text))]
len(text_tokens)

180

In [41]:
text_tokens[0]

['Apple',
 '’',
 's',
 'COO',
 'gives',
 'blistering',
 'testimony',
 'on',
 'Qualcomm',
 '’',
 's',
 '``',
 'onerous',
 'demands',
 "''",
 'over',
 'cellular',
 'standard-essential',
 '#',
 'patents',
 '$',
 'AAPL',
 '$',
 'QCOM',
 '@',
 'danieleran']

In [42]:
text_tokens = [[w for w in s if w not in stop_words] for s in text_tokens ]
len(text_tokens)

180

In [43]:
#nltk.download('maxent_ne_chunker')
#nltk.download('words')

In [44]:
import nltk
sentence = "At eight o'clock on Thursday morning arthur and microsoft didn't feel very good."
tokens = nltk.word_tokenize(sentence)
tagged = nltk.pos_tag(tokens)
entities = nltk.chunk.ne_chunk(tagged)
tagged

[('At', 'IN'),
 ('eight', 'CD'),
 ("o'clock", 'NN'),
 ('on', 'IN'),
 ('Thursday', 'NNP'),
 ('morning', 'NN'),
 ('arthur', 'NN'),
 ('and', 'CC'),
 ('microsoft', 'NN'),
 ('did', 'VBD'),
 ("n't", 'RB'),
 ('feel', 'VB'),
 ('very', 'RB'),
 ('good', 'JJ'),
 ('.', '.')]

In [45]:
tagged = [nltk.pos_tag(text) for text in text_tokens]

In [46]:
tagged[0][0][1]

'NNP'

In [49]:
tagged[0]

[('Apple', 'NNP'),
 ('’', 'NNP'),
 ('COO', 'NNP'),
 ('gives', 'VBZ'),
 ('blistering', 'VBG'),
 ('testimony', 'NN'),
 ('Qualcomm', 'NNP'),
 ('’', 'NNP'),
 ('``', '``'),
 ('onerous', 'JJ'),
 ('demands', 'NNS'),
 ("''", "''"),
 ('cellular', 'JJ'),
 ('standard-essential', 'JJ'),
 ('#', '#'),
 ('patents', 'NNS'),
 ('$', '$'),
 ('AAPL', 'NNP'),
 ('$', '$'),
 ('QCOM', 'NNP'),
 ('@', 'NNP'),
 ('danieleran', 'NN')]

In [50]:
[a for a in tagged[0] if 'NNP' in a]

[('Apple', 'NNP'),
 ('’', 'NNP'),
 ('COO', 'NNP'),
 ('Qualcomm', 'NNP'),
 ('’', 'NNP'),
 ('AAPL', 'NNP'),
 ('QCOM', 'NNP'),
 ('@', 'NNP')]

In [51]:
text_no_NE = [[s for s in w if 'NNP' not in s] for w in tagged]

In [52]:
text_no_NE[0]

[('gives', 'VBZ'),
 ('blistering', 'VBG'),
 ('testimony', 'NN'),
 ('``', '``'),
 ('onerous', 'JJ'),
 ('demands', 'NNS'),
 ("''", "''"),
 ('cellular', 'JJ'),
 ('standard-essential', 'JJ'),
 ('#', '#'),
 ('patents', 'NNS'),
 ('$', '$'),
 ('$', '$'),
 ('danieleran', 'NN')]

In [72]:
print(len(text_no_NE))
print(type(text_no_NE[0]))

180
<class 'list'>


In [73]:
text_no_NE = [[w[0] for w in mylist] for mylist in text_no_NE]
print(len(text_no_NE))
text_no_NE[0]

180


['gives',
 'blistering',
 'testimony',
 '``',
 'onerous',
 'demands',
 "''",
 'cellular',
 'standard-essential',
 '#',
 'patents',
 '$',
 '$',
 'danieleran',
 'https',
 ':',
 '//t.co/VMoYzzALR6',
 'https',
 ':',
 '//t.co/AVJxbyvsZP']

In [49]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(text)
X_train_counts = count_vect.fit_transform(aapl_text)

NameError: name 'aapl_text' is not defined

In [23]:
X_train_counts.shape

(180, 1706)

In [24]:
count_vect.vocabulary_.get(u'algorithm')

In [25]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)    
X_train_tfidf.shape

(180, 1706)

In [26]:
#tfidf_transformer.get_feature_names()

In [27]:
feature_names = count_vect.get_feature_names()
 
#get tfidf vector for first document
first_document_vector=X_train_tfidf[0]
 
#print the scores
ft = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"])
ft=ft.sort_values(by=["tfidf"],ascending=False)
ft.head(10)

,tfidf
vmoyzzalr6,0.256534
patents,0.256534
avjxbyvszp,0.256534
essential,0.256534
standard,0.256534
onerous,0.256534
demands,0.256534
qcom,0.256534
blistering,0.256534
coo,0.256534


In [28]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, tr.chg)

In [29]:
tr['pred'] = clf.predict(X_train_tfidf)
tr.head()

C:\Users\hadit\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,text,Date,name,chg,pred
33,Apple’s COO gives blistering testimony on Qual...,2019-01-15,AppleInsider,1,1
213,"Highlight: ""Laggards in today's session were p...",2019-12-03,Yahoo Finance,1,1
137,"Highlight: ""It's a very, very, very significan...",2019-07-03,Yahoo Finance,0,1
202,Highlight: “Early signs are that the iPhone 11...,2019-10-30,Yahoo Finance,1,1
101,Highlight: Goldman Sachs reiterated its Neutra...,2019-04-24,Yahoo Finance,0,1


In [30]:
from sklearn.metrics import confusion_matrix
confusion_matrix(tr.chg,tr.pred)

array([[ 22,  43],
       [  0, 115]], dtype=int64)

In [31]:
tr.chg.value_counts()

1    115
0     65
Name: chg, dtype: int64

In [15]:
#docs_new = ['God is love', 'OpenGL on the GPU is fast']
docs_new = 

In [16]:
X_new_counts = count_vect.transform(docs_new)

In [17]:
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

In [18]:
predicted = clf.predict(X_new_tfidf)
predicted

array([3, 1], dtype=int64)

In [19]:
type(predicted)

numpy.ndarray

In [20]:
len(predicted)

2

In [31]:
 for doc, category in zip(docs_new, predicted):
        print('%r => %s' % (doc, twenty_train.target_names[category]))

'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics
